In [135]:
import torch
import pandas as pd
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

from sklearn.metrics import accuracy_score
import numpy as np

In [136]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [137]:
data = pd.read_csv('diabetes.csv')
data_np = data.to_numpy()
tensor_data = torch.tensor(data_np)

rows, cols = tensor_data.size()

#=========================================================================
tensor_data_in_train = tensor_data[0:615,:-1].float()
tensor_data_out_train = tensor_data[:615, -1:].float()

train_dataset = TensorDataset(tensor_data_in_train, tensor_data_out_train)
#=========================================================================

tensor_data_in_test = tensor_data[615:, :-1].float()
tensor_data_out_test = tensor_data[615:, -1:].float()

test_dataset = TensorDataset(tensor_data_in_test, tensor_data_out_test)

In [138]:
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

In [139]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(0.5)
        self.layer1 = nn.Linear(8, 1)

    def forward(self, x):
        x = self.layer1(x)
        #x = self.sigmoid(x)
        #x = self.dropout(x)
        return x

In [140]:
model = NeuralNet().to(device)

criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.00001)

In [141]:
epochs = 30000
for epoch in range(epochs):
    for inputs, labels in train_loader:
        inputs, labels = inputs.float().to(device), labels.float().to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 10000 == 0:
        print(f"Epoch: {epoch + 1}/{epochs}, Loss: {loss.item():.3f}")

Epoch: 10000/30000, Loss: 0.150
Epoch: 20000/30000, Loss: 0.046
Epoch: 30000/30000, Loss: 0.182


In [142]:
correct, total = 0, 0

model.eval()
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.float().to(device), labels.float().to(device)
        outputs = model(inputs)
        predictions = torch.round(outputs) # округляем числа от 0 до 1
        total += labels.size(0)
        correct += (predictions == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy: {accuracy:.2f}%')

Accuracy: 73.86%


In [143]:
model.eval().cpu
model.cpu()

y_pred, y_true = [], []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.cpu(), labels.cpu()
        outputs = model(inputs)
        predictions = torch.round(outputs)
        y_pred.append(predictions.numpy())
        y_true.append(labels.numpy())

y_pred = np.concatenate(y_pred)
y_true = np.concatenate(y_true)

accuracy = accuracy_score(y_true, y_pred)
print(f"{accuracy * 100:.2f}%")

73.86%


In [144]:
model.load_state_dict(torch.load('model.pt'))

model.eval()
model.cpu()

y_pred, y_true = [], []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.cpu(), labels.cpu()
        outputs = model(inputs)
        predictions = torch.round(outputs)
        y_pred.append(predictions.numpy())
        y_true.append(labels.numpy())

y_pred = np.concatenate(y_pred)
y_true = np.concatenate(y_true)

accuracy = accuracy_score(y_true, y_pred)
print(f"{accuracy * 100:.2f}%")

75.16%


In [145]:
# Pregnancies, Glucose, BloodPressure, SkinThickness, Insulin, BMI, Diabetes Pedigree Function, Age

input_data = torch.tensor([8,183,64,0,0,23.3,0.672,32]).view(1, -1).cpu()
input_data = input_data.float()


prediction = model(input_data)
predicted = 'Positive' if prediction.item() >= 0.5 else 'Negative'
print(f"Prediction: {predicted}, {prediction.item()*100:.1f}%")

Prediction: Positive, 70.2%
